In [1]:
import pandas as pd
from datetime import datetime
import warnings
import matplotlib.pyplot as plt
import numpy as np
from urllib import parse


In [2]:
word1005 = pd.read_csv("1005Log.csv")
word1006 = pd.read_csv("1006Log.csv")
word1007 = pd.read_csv("1007Log.csv")
word1008 = pd.read_csv("1008Log.csv")
word1009 = pd.read_csv("1009Log.csv")
word1010 = pd.read_csv("1010Log.csv")
word1011 = pd.read_csv("1011Log.csv")

In [15]:
allCnt = pd.concat([word1005,word1006,word1007,word1008,word1009,word1010,word1011])
allCnt

,서버주소,요청시간,경로,HTTP 상태 코드,전송바이트(header제외)
0,172.23.204.13,[04/Oct/2021:00:00:01,cmp/main-pop-word/?limit=10&offset=0&FMY_SITE_...,200,64
1,172.23.204.13,[04/Oct/2021:00:00:01,cmp/main-pop-word/?limit=10&offset=0&FMY_SITE_...,200,64
2,172.23.204.13,[04/Oct/2021:00:00:01,cmp/main-pop-word/?limit=10&offset=0&FMY_SITE_...,200,1713
3,172.23.204.13,[04/Oct/2021:00:00:01,cmp/main-pop-word/?limit=10&offset=0&FMY_SITE_...,200,1563
4,172.23.204.13,[04/Oct/2021:00:00:01,cmp/main-pop-word/?limit=10&offset=0&FMY_SITE_...,200,1581
...,...,...,...,...,...
289093,172.23.204.24,[10/Oct/2021:23:59:58,h-community/?limit=2&offset=0&search_multi_mat...,200,3037
289094,172.23.204.24,[10/Oct/2021:23:59:58,mat-mov/?limit=10&offset=0&search=6회 펜토미노가 심쿵해...,200,12963
289095,172.23.204.24,[10/Oct/2021:23:59:59,cmp/ark-mid/?limit=10&offset=0&search=마이그래 HTT...,200,64
289096,172.23.204.24,[10/Oct/2021:23:59:59,mat-mov/?limit=10&offset=0&search=6회 돌도끼 수학 [1...,200,16574


In [4]:
#에러난 데이터만 모으기

error1005 = word1005[word1005["HTTP 상태 코드"] == 500]
error1006 = word1006[word1006["HTTP 상태 코드"] == 500]
error1007 = word1007[word1007["HTTP 상태 코드"] == 500]
error1008 = word1008[word1008["HTTP 상태 코드"] == 500]
error1009 = word1009[word1009["HTTP 상태 코드"] == 500]
error1010 = word1010[word1010["HTTP 상태 코드"] == 500]
error1011 = word1011[word1011["HTTP 상태 코드"] == 500]

In [5]:
error = pd.concat([error1005,error1006,error1007,error1008,error1009,error1010,error1011])

In [6]:
error = error.reset_index(drop=True)
error

,서버주소,요청시간,경로,HTTP 상태 코드,전송바이트(header제외)
0,172.23.204.23,[04/Oct/2021:00:10:38,j-course/?limit=10&offset=0&search=The 쉬운 영어독해...,500,19582
1,172.23.204.23,[04/Oct/2021:00:10:38,j-course-premium/?limit=10&offset=0&search=The...,500,19620
2,172.23.204.23,[04/Oct/2021:00:10:38,j-lecture/?limit=10&offset=0&search=The 쉬운 영어독...,500,19638
3,172.23.204.23,[04/Oct/2021:00:10:38,j-lecture-premium/?limit=10&offset=0&search=Th...,500,19626
4,172.23.204.24,[04/Oct/2021:00:15:51,j-course/?limit=10&offset=0&search=The 쉬운 영어독해...,500,19583
...,...,...,...,...,...
487,172.23.204.13,[10/Oct/2021:22:23:24,j-course/?limit=10&offset=0&search=쉬운영문빕 HTTP/1.1,500,19532
488,172.23.204.23,[10/Oct/2021:23:51:48,j-course/?limit=10&offset=0&search=The 쉬운 영문법 ...,500,19553
489,172.23.204.23,[10/Oct/2021:23:51:48,j-course-premium/?limit=10&offset=0&search=The...,500,19591
490,172.23.204.23,[10/Oct/2021:23:51:48,j-lecture/?limit=10&offset=0&search=The 쉬운 영문법...,500,19557


In [7]:
dfList = []
for i in range(len(error)):
    #서버주소 
    addr = error["서버주소"][i]
    #검색기록 없으면 nan처리
    if error["경로"].str.find('search=')[i] == -1 and error["경로"].str.find('search_multi_match=')[i] == -1 :
        error["경로"][i] = np.nan
    #검색이 search = , search_multi_match= 두가지 경우가 있음
    else:
        #url만 자르기
        endPoint = int(error["경로"].str.find("?")[i])
        url = error["경로"][i][:endPoint]
        
        #검색기록만 자르기
        if int(error["경로"].str.find('search=')[i]) == -1:
            start = int(error["경로"].str.find('search_multi_match=')[i] +19)
        else:
            start = int(error["경로"].str.find('search=')[i] + 7)
        error["경로"][i] = error["경로"][i][start:]
    
        if int(error["경로"].str.find("&")[i]) == -1:
            end = int(error["경로"].str.find(" HTTP/1.1")[i])
        else:
            end = int(error["경로"].str.find("&")[i])
        keyword = error["경로"][i][:end]
    
        df = pd.DataFrame({
            "서버주소":addr,
            "url":url,
            "검색어":keyword 
            },index = [i])
        
        dfList.append(df)
        
errorFinal = pd.concat(dfList)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [8]:
errorFinal

,서버주소,url,검색어
0,172.23.204.23,j-course/,The 쉬운 영어독해 Ⅰ
1,172.23.204.23,j-course-premium/,The 쉬운 영어독해 Ⅰ
2,172.23.204.23,j-lecture/,The 쉬운 영어독해 Ⅰ
3,172.23.204.23,j-lecture-premium/,The 쉬운 영어독해 Ⅰ
4,172.23.204.24,j-course/,The 쉬운 영어독해 Ⅰ
...,...,...,...
487,172.23.204.13,j-course/,쉬운영문빕
488,172.23.204.23,j-course/,The 쉬운 영문법 Ⅰ
489,172.23.204.23,j-course-premium/,The 쉬운 영문법 Ⅰ
490,172.23.204.23,j-lecture/,The 쉬운 영문법 Ⅰ


In [9]:
# 비율확인
len(errorFinal) / len(allCnt)

0.00020900798945784092

In [10]:
# 500에러가 어떤 경우에 가장 많이 나는지 확인 (행기준)
errorFinal.value_counts()[:50]

서버주소           url                 검색어                          
172.23.204.24  j-course/           The 쉬운 영문법 Ⅰ                     7
172.23.204.13  j-lecture-premium/  The 쉬운 영문법 Ⅰ                     6
172.23.204.24  j-course/           The 쉬운 영문법                       6
               j-course-premium/   The 쉬운 영문법 Ⅰ                     6
172.23.204.13  j-lecture/          The 쉬운 영문법 Ⅰ                     6
172.23.204.24  j-lecture/          The 쉬운 영문법 Ⅰ                     5
172.23.204.13  j-course-premium/   The 쉬운 영문법 Ⅰ                     5
172.23.204.24  j-lecture-premium/  The 쉬운 영문법 Ⅰ                     5
               j-course-premium/   The 쉬운 영문법                       5
172.23.204.13  j-course/           The 쉬운 영문법 Ⅰ                     5
172.23.204.24  j-lecture/          The 쉬운 영문법                       5
               j-lecture-premium/  The 쉬운 영문법                       4
172.23.204.23  j-lecture/          The 쉬운 영어듣기                      4
               j-lecture-

In [12]:
# 500에러가 개별변수로 가장 많이 나는지 확인 (열기준)

#서버주소 별 확인
errorFinal['서버주소'].value_counts()

172.23.204.13    222
172.23.204.24    166
172.23.204.23     91
172.23.204.14     13
Name: 서버주소, dtype: int64

In [45]:
#전체데이터 기준 서버주소 172.23.204.13 에러비율 확인 
len(errorFinal[errorFinal['서버주소']== "172.23.204.13"]) / len(allCnt[allCnt["서버주소"] == "172.23.204.13"])

0.00023594129950516094

In [13]:
# url 별 확인
errorFinal['url'].value_counts()

j-course/             111
j-course-premium/      96
j-lecture/             96
j-lecture-premium/     95
p-lecture/             48
p-course/              46
Name: url, dtype: int64

In [42]:
#전체데이터 기준 url j-course/ 에러비율 확인 
len(errorFinal[errorFinal['url']== "j-course/"]) / len(allCnt[allCnt["경로"] .str.contains("j-course/")])

0.0050388124744654775

In [14]:
errorFinal['검색어'].value_counts()

The 쉬운 영문법 Ⅰ                  55
The 쉬운 영문법                    36
The 쉬운 영어듣기                   25
중학수학 뜀틀 개념편 중1(상)(2021년)      21
더쉬운영문법                        19
                              ..
정승익 더 쉬운 영문                    1
더쉬운영어듣기                        1
EBSMath 4학년 2학기 분수의 덧셈과 뺄셈     1
중학 과학 끝장 내기                    1
개코수학                           1
Name: 검색어, Length: 96, dtype: int64

In [39]:
#전체 데이터중 The 쉬운 영문법 Ⅰ 에러 비율 확인
len(errorFinal[errorFinal['검색어']== "The 쉬운 영문법 Ⅰ"]) / len(allCnt[allCnt["경로"] .str.contains("The 쉬운 영문법 Ⅰ")])


0.25821596244131456

In [40]:
#전체 데이터중 "The 쉬운" 이 포함된 검색어 에러 비율 확인
len(errorFinal[errorFinal["검색어"] .str.contains("The 쉬운")]) / len(allCnt[allCnt["경로"] .str.contains("The 쉬운")])

0.20823798627002288